In [31]:
from pyspark import SparkContext

In [32]:
sc = SparkContext(master="local", appName="AccionesTransformaciones")

In [33]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [34]:
rdd1.collect()

[1, 2, 3]

In [35]:
sc

<SparkContext master=local appName=AccionesTransformaciones>

In [36]:
path = "files/"

equipos_olimpicos_rdd = sc.textFile(path+"paises.csv") \
                          .map(lambda line: line.split(','))

In [37]:
equipos_olimpicos_rdd.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [38]:
equipos_olimpicos_rdd.map(lambda x: x[2]) \
                      .distinct() \
                      .count()

231

In [39]:
equipos_olimpicos_rdd.map(lambda x: (x[2],x[1])) \
                      .groupByKey() \
                      .mapValues(len) \
                      .take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [40]:
equipos_olimpicos_rdd.map(lambda x: (x[2],x[1])) \
                      .groupByKey() \
                      .mapValues(list) \
                      .take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [41]:
equipos_argentinos = equipos_olimpicos_rdd.filter(lambda x: "ARG" in x)
equipos_argentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [42]:
equipos_olimpicos_rdd.count()

1185

In [43]:
equipos_olimpicos_rdd.countApprox(20)

1185

In [44]:
deportista_olimpico = sc.textFile(path+"deportista.csv") \
                        .map(lambda l: l.split(','))

deportista_olimpico2 = sc.textFile(path+"deportista2.csv") \
                        .map(lambda l: l.split(','))

In [45]:
deportista_olimpico = deportista_olimpico.union(deportista_olimpico2)

In [46]:
deportista_olimpico.count()

135572

In [47]:
equipos_olimpicos_rdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [48]:
deportista_olimpico.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [49]:
deportista_olimpico.map(lambda l: [l[-1], l[:-1]]) \
                  .join(equipos_olimpicos_rdd.map(lambda x: [x[0], x[2]])) \
                  .takeSample(False, 5, 25)

[('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('154', (['39161', 'Angel Merdzhanov Gavrilov', '1', '24', '0', '0'], 'BUL')),
 ('1084',
  (['62843', 'Olha Vasylivna Korobka', '2', '18', '181', '167'], 'UKR')),
 ('678', (['97550', 'Puntsagiin Skhbat', '1', '24', '174', '82'], 'MGL')),
 ('1096', (['106789', 'Hugo Scherzer', '1', '43', '0', '0'], 'USA'))]

In [50]:
resultado = sc.textFile('files/resultados.csv').map(lambda x: x.split(','))

In [51]:
resultado = resultado.filter(lambda l: 'NA' not in l[1])

In [52]:
resultado.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [53]:
deportista_paises = deportista_olimpico.map(lambda l: [ l[-1], l[:-1] ]) \
  .join( equipos_olimpicos_rdd.map( lambda x: (x[0], x[2]) ) )
deportista_paises.take(2)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN'))]

In [54]:
deportista_paises.join( resultado ).take(2)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold'))]

In [55]:
valores_medalals = {
  'Gold': 7,
  'Silver': 5,
  'Bronze': 4
}

paises_medallas = deportista_paises.join(resultado)

In [60]:
paises_medallas = paises_medallas.map(lambda x: [x[1][0][-1], valores_medalals[x[1][1]]])

In [62]:
from operator import add

conclusion = paises_medallas.reduceByKey(add) \
  .sortBy(lambda x: x[1], ascending=False)

In [66]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

In [57]:
#sc.stop()